## 读取数据集

In [12]:
training_d = pd.read_csv('./dataset-20211103/training.csv')
test_d = pd.read_csv('./dataset-20211103/test.csv')

In [13]:
training_d.head()
#test_d.head()

,buying,maint,doors,persons,lug_boot,safety,evaluation
0,high,low,5more,2,med,med,unacc
1,med,low,5more,4,small,low,unacc
2,med,med,4,more,med,low,unacc
3,low,med,3,more,med,low,unacc
4,low,vhigh,3,more,big,low,unacc


In [14]:
test_d.shape

(479, 6)

In [15]:
# for native_bayes
dic = {}
for i in test_d.values:
    out, precise = native_bayes_train(training_d, 'evaluation', i)
    if out =='unacc':
        if 'unacc' not in dic:
            dic['unacc']=1
        else:
            dic['unacc']+=1
    else:
        if 'acc' not in dic:
            dic['acc']=1
        else:
            dic['acc']+=1

In [ ]:
dic['unacc']/(dic['unacc']+dic['acc']),dic['acc']/(dic['unacc']+dic['acc'])

In [ ]:
dic

## 朴素贝叶斯实现

In [3]:
import pandas as pd
import numpy as np
import random

In [4]:
def native_bayes_train(df, labels_column_name,tuples_):
    '''this function is return the needed value of calculate 
    bayes function 
    df is the dataframe without identical columns'''
    #first get the labels,默认为分类问题
    #get the num of total record
    total_record = len(df)
    #create a dic to solve the corresponding value
    # the prior probability
    dic = {}
    for i in df.groupby(labels_column_name).count().itertuples():
        dic[i[0]] = [(i[1]+1)/(total_record+len(df.groupby(labels_column_name).count())),i[1]]
    # using all the feature except labels to cal the conditional independent values
    #print(dic)
    dic_2={}
    for i in dic.keys():
        df_1 = df[df[labels_column_name]==i]
        #where a is the conditional independence of given class, cal the x
        a = 1
        for x,j in enumerate(df_1.drop(labels_column_name, axis=1).iteritems()):
            temp=df_1.groupby(j[0]).count()
            #get the each value pf calculation
            if tuples_[x] not in temp.index:
                #using the lapulasi regularization
                a*= 1/(dic[i][1]+len(temp))
            else:
                a*=(temp.loc[tuples_[x]][1]+1)/(dic[i][1]+len(temp))
        dic_2[i]= a*dic[i][0]
    return max(dic_2,key = dic_2.get),dic_2

## 实现感知机
需要：
1. 输入矩阵
2. 权重矩阵(多少个神经元)
3. 激活函数
4. 输出结果
5. 反向传播，此处为反馈给权重矩阵

## KNN实现


需要：
1. 对n个attribute，所有的数据被视为n维空间的一个点
2. 任意两个点，我们说其“相近”，即称有small distance base on distance metric
3. 一般来说，度量通常使用l2范数（即欧氏距离）
4. 对k-nearest neighbour， 即找范围内k个最近的点
5. 选择第最多的个数作为你的预测class
6. 假设对应的features相同，则将距离为0，否则为1（即越多越远）
7. 对高维度的情况下，数据会出现过于稀疏的情况，此时我们需要增大数据量或者降维操作
8. 此时高维的情况，可以通过加权来对重要的attributes进行处理
9. k是一个超参数，可以通过交叉验证来处理
10. 测试时，给予权重 w = 1/(d(xi,xq)), 距离越近，权重越大，即不直接根据个数比较


In [ ]:
def knn(df_train, df_test,labels_column_name,k=3):
    '''
    k: nearest k number
    df: dataframe 
    df_test: the test data
    labels_coulmn_name: labels name
    '''
    labels = df_train[labels_column_name]
    data_size = df_train.shape[0]#total num of features
    #cal each point in testing data set to the train dataset
    # make sure that all the columns change into 1 and 0
    temp = pd.concat([df_train.drop(labels_column_name,axis=1),df_test])
    temp= pd.get_dummies(temp)
    train_mt = temp[:data_size].values
    test_mt = temp[data_size:].values
#     print(train_mt.shape,test_mt.shape)
    # cal the euclidiean distance
    dic ={}
    for i in test_mt:
        temp={}
        subtract_m = np.tile(i,(data_size,1)) - train_mt
        #suqare and sum
        subtract_m=subtract_m**2
        sum_m = np.sum(subtract_m, axis=1)
        fin_m=np.sqrt(sum_m)
        #print(fin_m.shape)
        #then return the index of the maximum
        sorted_index=np.argsort(fin_m)
        #print(sorted_index)
        # the kth minimum distance
        for i in range(k):
            # get the label index in labels
            if labels[sorted_index[i]] not in temp:
                temp[labels[sorted_index[i]]]=1
            else:
                temp[labels[sorted_index[i]]]+=1
        if max(temp,key = temp.get) not in dic:
            dic[max(temp,key = temp.get)]= 1
        else:
            dic[max(temp,key = temp.get)]+= 1
    return dic

In [ ]:
training_d.shape,test_d.shape

In [ ]:
dic2={}

In [ ]:
dic2=knn(training_d,test_d, labels_column_name='evaluation')
dic2

In [ ]:
dic2['unacc']/(dic2['unacc']+dic2['acc']),dic2['acc']/(dic2['unacc']+dic2['acc'])

In [ ]:
training_d

In [ ]:
test_d